In [1]:
import scipy.special as sp
import numpy as np
import uproot
import csv

import Utilities.Constants as Constants
import Utilities.Functions as Functions
from Utilities.ipython_exit import exit

print("Success")

Success


## Equation to replicate
The first attempt comes from an equation in the HNL to e+e- recast of HPS decays in MicroBooNE paper: https://arxiv.org/abs/2106.06548

This has an equation for Majorana HNLs with the HNL mass << muon mass.

In [2]:
s_W_squared = 0.231 #sin squared Weinberg, Taken from Wikipedia, rough
G_F = 1.1663787e-5 #GeV-2 Fermi couplingconst, Taken from Jinst

In [5]:
# Define a function here, which takes the variables U_mu4 and HNL mass.
# Doing all in GeV

N_mass = 0.1 #100 MeV
U_mu4 = 1e-4
Decay_width = (2*(G_F**2)*(U_mu4**2)*(N_mass**5)/(768*np.pi**3))*(1-4*s_W_squared+8*s_W_squared**2)

print(Decay_width)

5.746052508123807e-28
